<a href="https://colab.research.google.com/github/SahanaTP/Carbon_Assesment_and_Measurement_in_Agriculture/blob/main/code/data_collection/OCO2_Data_Collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


In [27]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=7121589c93e7fc8373ea2afc66538a80b046d80d5c0090984e93b70a7213f542
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [67]:
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import netCDF4 as nc

# converting the datetime format
from datetime import datetime


In [131]:
 !wget -p /content/oco2 --user sahana.thoravalliprabhuswamy --password Gr@sshopper7 -i /content/subset_OCO2_L2_Lite_FP_11.1r_20240414_101220_.txt

Streaming output truncated to the last 5000 lines.
Connecting to data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)|13.226.210.125|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://d2b3c3wh8s6en5.cloudfront.net/s3-1f9855127b9284f7b00b31754cfd6e15/gesdisc-cumulus-prod-protected.s3.us-west-2.amazonaws.com/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2022/oco2_LtCO2_220104_B11100Ar_230609002409s.nc4?A-userid=sahana.thoravalliprabhuswamy&Expires=1713093231&Signature=rNvu-u0yNWoy72VvUZcHBDYkcs0aYBwzzzc9hZqpLwMzpzVbUlSFWdljPsFo02cVpnKNhVEIaLq2tGZ5sYoC9pTnDT1cpg4mitOskQVMkR5rqdUzvcN3Yl8mI2BYEfsrTBceTqG8jWrxiOzTn5Z4OLB-nynz2gedMgWiND3Hu0hlpUXkNzxGItmIlxUZp5TIyobDq7UR~DhTvBF2K7OFOHXx6k9fssusUf-6LCC~CNfZ37WdYt1mdeH5gv4LHtUFzGOJaj2hI~tqmnkdbApDmgXVMWnc6ULqnFvTmEE7wNdd~FCKZjqi-Fc~71bRT76JBj21gv18KrRxyudqIn74YA__&Key-Pair-Id=KWW1M1QBBXTEK [following]
--2024-04-14 10:13:51--  https://d2b3c3wh8s6en5.cloudfront.net/s3-1f9855127b9284f7b00b31754cfd6e15/gesdisc-

In [14]:
#df_xco2= nc.Dataset('/content/cams73_latest_co2_flux_surface_inst_202201.nc')

In [5]:
# DATE time function
def conv_date(d):
#     return datetime.strptime(str(d), '%Y%m%d%H%M%S%f')
    return datetime.utcfromtimestamp(d) ## dateTime conversion

In [72]:
def convHdf(path_file, n=0):

    data= nc.Dataset(path_file)

    # get the HDF data and convert to CSV
    df_xco2= pd.DataFrame()

    df_xco2['Xco2']= data.variables['xco2'][:]
    df_xco2['Latitude']= data.variables['latitude'][:]
    df_xco2['Longitude']= data.variables['longitude'][:]
    df_xco2['quality_flag']= data.variables['xco2_quality_flag'][:]

    # Date
    #df_xco2['DateTime']= data.variables['sounding_id'][:]

    ## DateTime variable name changed in new ver.; `sounding_id` -> `time`
    df_xco2['DateTime']= data.variables['time'][:]

    #Convert soundingID to datetime format
    df_xco2['DateTime']= df_xco2['DateTime'].apply(conv_date)
    df_xco2['DateTime']= pd.to_datetime(df_xco2['DateTime'])

    # YEAR and month column
    df_xco2['Year']= df_xco2['DateTime'].dt.year
    df_xco2['Month']= df_xco2['DateTime'].dt.month
    df_xco2['Day']= df_xco2['DateTime'].dt.day

    # Refine the ENTIRE dataframe by GOOD quality_flag->0
    # NOTE: REDUCES the size of the file
    df_xco2= df_xco2[df_xco2['quality_flag'] == 0]


    date= str(data.variables['sounding_id'][0])

    print(df_xco2.head())
    # create a CSV and store on new folder: csv_files
    df_xco2_new = df_xco2[(df_xco2['Latitude']>36) & (df_xco2['Latitude']<38) & (df_xco2['Longitude']>118) & (df_xco2['Longitude']<121)]

    return df_xco2_new

In [106]:
convHdf('/content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2023/oco2_LtCO2_230101_B11100Ar_230609171456s.nc4')

           Xco2   Latitude   Longitude  quality_flag  \
164  413.748962 -75.030762 -146.272598             0   
172  413.726654 -75.000008 -146.391754             0   
173  412.980133 -75.002159 -146.460220             0   
176  413.825592 -74.978607 -146.282562             0   
177  413.496246 -74.981010 -146.349640             0   

                      DateTime  Year  Month  Day  
164 2023-01-01 00:46:20.921026  2023      1    1  
172 2023-01-01 00:46:21.615007  2023      1    1  
173 2023-01-01 00:46:21.642003  2023      1    1  
176 2023-01-01 00:46:21.894989  2023      1    1  
177 2023-01-01 00:46:21.921020  2023      1    1  


,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day


In [44]:
data_co2 = pd.read_csv('/content/df_xco2.csv')

In [58]:
data_co2[(data_co2['Latitude']>36) & (data_co2['Latitude']<38) & (data_co2['Longitude']>118)]

,Unnamed: 0,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day
16497,23301,414.75308,36.181305,138.17264,0,2021-01-01 03:58:58.921018,2021,1,1
16498,23302,412.33447,36.174526,138.18172,0,2021-01-01 03:58:58.948014,2021,1,1
16499,23304,416.57913,36.160862,138.19960,0,2021-01-01 03:58:59.002007,2021,1,1
16500,23306,415.15402,36.194626,138.17587,0,2021-01-01 03:58:59.281990,2021,1,1
16501,23307,413.34900,36.187790,138.18489,0,2021-01-01 03:58:59.308986,2021,1,1
16502,23308,412.04556,36.167100,138.21130,0,2021-01-01 03:58:59.389010,2021,1,1
16503,23315,413.16257,36.428340,138.09271,0,2021-01-01 03:59:02.895006,2021,1,1
16504,23316,412.04907,36.421650,138.10187,0,2021-01-01 03:59:02.920996,2021,1,1
16505,23321,410.68390,36.474586,138.09322,0,2021-01-01 03:59:03.947987,2021,1,1


In [132]:
# Path to the directory containing the files
directory_path = '/content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2022'

# List all files in the directory
files = os.listdir(directory_path)

data_xco2_2021 = pd.DataFrame(columns=['Unnamed: 0','Xco2','Latitude','Longitude','quality_flag',	'DateTime',	'Year',	'Month',	'Day'])

# Iterate over each file in the directory
for file_name in files:
  if os.path.isfile(os.path.join(directory_path, file_name)) and file_name.endswith('.nc4'):
        # Construct the full path to the CSV file
        file_path = os.path.join(directory_path, file_name)
        data_xco2_2021 = pd.concat([data_xco2_2021, convHdf(file_path)], axis=0, ignore_index=True)

          Xco2   Latitude   Longitude  quality_flag  \
54  415.335083 -32.027866 -161.627319             0   
58  415.463135 -32.009892 -161.631287             0   
59  415.304932 -32.004307 -161.652130             0   
63  415.614197 -31.991980 -161.635315             0   
64  415.324890 -31.986408 -161.656158             0   

                     DateTime  Year  Month  Day  
54 2022-07-13 00:33:52.363026  2022      7   13  
58 2022-07-13 00:33:52.695995  2022      7   13  
59 2022-07-13 00:33:52.722992  2022      7   13  
63 2022-07-13 00:33:53.029005  2022      7   13  
64 2022-07-13 00:33:53.056001  2022      7   13  
          Xco2   Latitude   Longitude  quality_flag  \
4   416.622192 -31.134371 -166.460892             0   
12  415.015137 -30.898037 -166.606628             0   
13  415.782532 -30.893829 -166.626022             0   
18  415.606476 -30.803520 -166.573212             0   
19  414.928253 -30.796150 -166.519791             0   

                     DateTime  Year  M

In [133]:
data_xco2_2021.drop(columns='Unnamed: 0', inplace = True)
data_xco2_2021.drop(columns='quality_flag', inplace = True)
# Convert timestamp_column to datetime object
data_xco2_2021['date_time'] = pd.to_datetime(data_xco2_2021['DateTime'])

# Extract date component from datetime object
data_xco2_2021['date_column'] = data_xco2_2021['date_time'].dt.date
data_xco2_2021.drop(columns=['DateTime', 'date_time'], inplace = True)
data_xco2_2021.sort_values(by= 'date_column', inplace = True)
data_xco2_2021.reset_index(drop=True, inplace=True)
desired_order = ['date_column', 'Latitude',	'Longitude',	'Xco2', 'Year',	'Month',	'Day']
data_xco2_2021 = data_xco2_2021[desired_order]
data_xco2_2021

,date_column,Latitude,Longitude,Xco2,Year,Month,Day
0,2022-01-01,37.167229,120.908844,419.526978,2022,1,1
1,2022-01-01,37.267155,120.879578,417.437134,2022,1,1
2,2022-01-01,37.273979,120.870392,418.307556,2022,1,1
3,2022-01-01,37.280739,120.861130,418.572906,2022,1,1
4,2022-01-01,37.287415,120.851830,418.282318,2022,1,1
...,...,...,...,...,...,...,...
6306,2022-12-28,37.304699,118.423126,422.726074,2022,12,28
6307,2022-12-28,37.291168,118.435875,423.613525,2022,12,28
6308,2022-12-28,37.263737,118.461037,423.894318,2022,12,28
6309,2022-12-28,37.235615,118.485695,424.208679,2022,12,28


In [134]:
data_xco2_2021.to_csv(r'/content/data_xco2_2022.csv', index = False)

In [96]:
# prompt: zip a folder and download

!zip -r /content/data_xco2_2022_original.zip /content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r
from google.colab import files
files.download('/content/data_xco2_2022_original.zip')


  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/ (stored 0%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2020/ (stored 0%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_201231_B11100Ar_230604004338s.nc4 (deflated 2%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2022/ (stored 0%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2022/oco2_LtCO2_220101_B11100Ar_230609001844s.nc4 (deflated 2%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2021/ (stored 0%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2021/oco2_LtCO2_210312_B11100Ar_230607182045s.nc4 (deflated 2%)
  adding: content/data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2021/oco2_LtCO2_210418_B11100Ar_230607193130s

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>